In [ ]:
import pymysql
import pandas as pd
import passwords
import sys

mysqlip = passwords.mysqlip
mysqlpassword = passwords.mysqlpassword
mysqlusername = passwords.mysqlusername
mysqldb = passwords.mysqldb

print(mysqlip)
counties = [
    'lawrence',
    'butler',
    'beaver',
    'allegheny'
]

In [ ]:
def sendSQLCommand(sql):
    try:
        connection = pymysql.connect(host=mysqlip,
                                     user=mysqlusername,
                                     password=mysqlpassword,
                                     db=mysqldb,
                                     charset='utf8mb4',
                                     cursorclass=pymysql.cursors.DictCursor)

        cursor = connection.cursor()
        cursor.execute(sql)
        dockets = cursor.fetchall()
        cursor.close()
        return dockets
    except Exception as e:
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)

In [ ]:
#Charges Eligible for Expungement

#Just Non Conviction Charges Eligible
print("Non Conviction Charges")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM pennsylvania2022.expungement WHERE DispositionCategory= 'Non-Conviction'
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
display(df)

title = 'Charges Eligible for Expungement Because of Non-Conviction'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)
temp.figure.savefig("Expungement Graphs/" + title + ".png",bbox_inches="tight")

#Summary Convictions Eligible
print("Summary Convictions Eligible")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE 
ChargeGrade = 's' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL)
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)

title = 'Charges Eligible for Expungement Because of Summary Conviction'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)
temp.figure.savefig("Expungement Graphs/" + title + ".png",bbox_inches="tight")

#Over 70 Convictions Eligible
print("Over 70 Convictions Eligible")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE 
Age >= 70 and TenYearArrestFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL)
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)

title = 'Charges Eligible for Expungement Because of Age Over 70'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)
temp.figure.savefig("Expungement Graphs/" + title + ".png",bbox_inches="tight")

#All Charges Eligible
print("All Eligible Charges")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE 
DispositionCategory= 'Non-Conviction'
or (ChargeGrade = 's' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (ARDCompletion = True and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (Age >= 70 and TenYearArrestFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)

title = 'Total Charges Eligible for Expungement'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)
temp.figure.savefig("Expungement Graphs/" + title + ".png",bbox_inches="tight")


In [ ]:
#Charges currently eligible for expungement

#Charges Eligible for Expungement Under the Non Conviction Rule
print("Charges Eligible Non-Conviction")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE DispositionCategory= 'Non-Conviction'
GROUP BY County'''
cases_eligible_nonconviction = sendSQLCommand(sql)
print(cases_eligible_nonconviction)
df_nonconvictions = pd.DataFrame(cases_eligible_nonconviction)
df_nonconvictions['County'] = df_nonconvictions['County'].str.capitalize()
display(df_nonconvictions)
title = 'Charges Eligible for Expungement Because of Summary Conviction'
temp = df_nonconvictions.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

#Charges Eligible for Expungement Under the Summary Conviction Rule
print("Charges Eligible Non-Conviction")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE ChargeGrade = 'S' and 
DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL)
GROUP BY County'''
cases_eligible_summary_conviction = sendSQLCommand(sql)
print(cases_eligible_summary_conviction)
df_summary_convictions = pd.DataFrame(cases_eligible_summary_conviction)
df_summary_convictions['County'] = df_summary_convictions['County'].str.capitalize()
display(df_summary_convictions)
title = 'Charges Eligible for Expungement Because of Summary Conviction'
temp = df_summary_convictions.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)


# #Charges Eligible for Expungement Under ARD
# print("Charges Eligible Summary Convivction")
# sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE 
# ARDCompletion = 'ARD Completed' and (FinesAndFees = 0 or FinesAndFees IS NULL)
# GROUP BY County'''
# cases_eligible_ard = sendSQLCommand(sql)
# print(cases_eligible_ard)
# df_ard = pd.DataFrame(cases_eligible_ard)
# df_ard['County'] = df_ard['County'].str.capitalize()
# display(df_ard)
# title = 'Charges Eligible for Expungement Because of ARD Completion'
# temp = df_ard.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)


#Charges Eligible for Expungement Because of Age
print("Charges Eligble M2, M3, and M")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE 
Age >= 70 and TenYearArrestFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL)
GROUP BY County'''
cases_eligible_age = sendSQLCommand(sql)
df_age = pd.DataFrame(cases_eligible_age)
df_age['County'] = df_age['County'].str.capitalize()
display(df_age)
title = 'Charges Eligible for Expungement Becasue Defendant Over 70'
temp = df_age.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

#Charges Eligible for Expungement ALL
print("Charges Eligble All")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM expungement WHERE 
DispositionCategory= 'Non-Conviction'
or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (ARDCompletion = 'ARD Completed' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (Age >= 70 and TenYearArrestFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Currently Eligible for Expungement'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)
temp.figure.savefig("Sealing Graphs/" + title + ".png",bbox_inches="tight")


#There is probably a better way to add all the sql results but keep their eligibility category
final = [
    {'County':'lawrence'},
    {'County':'butler'},
    {'County':'beaver'},
    {'County':'allegheny'}
]
for item in cases_eligible_nonconviction:
    for x,item2 in enumerate(final):
        if(item['County'] == item2['County']):
            final[x]['Non-Convictions'] = item['Charges']

# for item in cases_eligible_ard:
#     for x,item2 in enumerate(final):
#         if(item['County'] == item2['County']):
#             final[x]['Eligible ARD Convictions'] = item['Charges']

for item in cases_eligible_summary_conviction:
    for x,item2 in enumerate(final):
        if(item['County'] == item2['County']):
            final[x]['Eligible Summary Convictions'] = item['Charges']

for item in cases_eligible_age:
    for x,item2 in enumerate(final):
        if(item['County'] == item2['County']):
            final[x]['Eligible Over 70 Convictions'] = item['Charges']
print(final)

df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Eligible for \nExpungement By Eligibilty Category'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",stacked=True,x='County',ylabel='Amount of Charges',title=title)
temp.figure.savefig("Expungement Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler

df = df[df['County'] != 'Allegheny']
display(df)
title = 'Charges Eligible for Expungement By Eligibilty \nCategory in Lawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",stacked=True,x='County',ylabel='Amount of Charges',figsize = (9,5),title=title)
temp.figure.savefig("Expungement Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

In [ ]:
#Cases and Charges Currently Eligible for Expungement By Charge/Case/person
print("Charges Eligible Expungement")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
DispositionCategory= 'Non-Conviction'
or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (ARDCompletion = 'ARD Completed' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (Age >= 70 and TenYearArrestFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
THEN 1
ELSE 0
END AS Eligibility
FROM expungement) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
DispositionCategory= 'Non-Conviction'
or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (ARDCompletion = 'ARD Completed' and (FinesAndFees = 0 or FinesAndFees IS NULL))
or (Age >= 70 and TenYearArrestFreePeriod = 'True' and (FinesAndFees = 0 or FinesAndFees IS NULL))
THEN 1
ELSE 0
END AS Eligibility
FROM expungement) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Record Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Record Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges and Cases Eligible for Expungement'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Charges, Cases, and Records',title=title)
temp.figure.savefig("Expungement Graphs/" + title +".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Charges and Cases Eligible for Expungement in \nLawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Charges, Cases, and Records',title=title)
temp.figure.savefig("Expungement Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Charges/Cases with at least one charge/Full Cases Eligible for expungement but for fine and fee
print("Charges and Cases Eligible But For Fine and Fee")

sql = '''SELECT *,COUNT(*) as charge_count FROM expungement WHERE 
(ChargeGrade = 'S' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and FinesAndFees > 0)
or (ARDCompletion = 'ARD Completed' and FinesAndFees > 0)
or (Age >= 70 and TenYearArrestFreePeriod = 'True' and FinesAndFees > 0)
GROUP BY DocketNumber'''

finalanswer = []

dockets = sendSQLCommand(sql)
charges_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
at_least_1_cases_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
full_cases_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
for docket in dockets:
    if(float(docket['FinesAndFees']) <= 100):
        charges_buckets['$100 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$100 or Less'] += 1
        else:
            at_least_1_cases_buckets['$100 or Less'] += 1

    if(float(docket['FinesAndFees']) <= 250):
        charges_buckets['$250 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$250 or Less'] += 1
        else:
            at_least_1_cases_buckets['$250 or Less'] += 1

    if(float(docket['FinesAndFees']) <= 500):
        charges_buckets['$500 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$500 or Less'] += 1
        else:
            at_least_1_cases_buckets['$500 or Less'] += 1
        
    if(float(docket['FinesAndFees']) <= 1000):
        charges_buckets['$1000 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$1000 or Less'] += 1
        else:
            at_least_1_cases_buckets['$1000 or Less'] += 1

    
    if(float(docket['FinesAndFees']) <= 5000):
        charges_buckets['$5000 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$5000 or Less'] += 1
        else:
            at_least_1_cases_buckets['$5000 or Less'] += 1
    
    charges_buckets["All"] += docket['charge_count']
    if(docket['charge_count'] == docket['TotalCharges']):
        full_cases_buckets['All'] += 1
    else:
        at_least_1_cases_buckets["All"] += 1


print(charges_buckets)
print(at_least_1_cases_buckets)
print(full_cases_buckets)

#Get Full Record
#Full Record Eligible But For Fee
sql = '''WITH temp AS 
(SELECT *, CASE 
WHEN 
(ChargeGrade = 'S' and DispositionCategory = 'Conviction' and FiveYearFreePeriod = 'True' and FinesAndFees > 0)
or (ARDCompletion = 'ARD Completed' and FinesAndFees > 0)
or (Age >= 70 and TenYearArrestFreePeriod = 'True' and FinesAndFees > 0)
THEN 1
ELSE 0
END AS Eligibility
FROM expungement) SELECT *, MIN(Eligibility) as CaseEligibility,IF(MIN(Eligibility) = 1,1,NULL) as `Record Fully Eligible` FROM temp GROUP BY PersonID
'''

finalanswer = []

dockets = sendSQLCommand(sql)
full_record_bucket = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}

for docket in dockets:
    if(docket['CaseEligibility'] == 1):
        if(float(docket['FinesAndFees']) <= 100):
            full_record_bucket['$100 or Less'] += 1

        if(float(docket['FinesAndFees']) <= 250):
            full_record_bucket['$250 or Less'] += 1


        if(float(docket['FinesAndFees']) <= 500):
            full_record_bucket['$500 or Less'] += 1

        if(float(docket['FinesAndFees']) <= 1000):
            full_record_bucket['$1000 or Less'] += 1


        if(float(docket['FinesAndFees']) <= 5000):
            full_record_bucket['$5000 or Less'] += 1

        full_record_bucket["All"] += 1






#Combine Both the Cases and Full Record Eligibility
for key,value in at_least_1_cases_buckets.items():
    finalanswer.append({"Amount":key,"Charges Eligible":charges_buckets[key],"Cases Partially Eligible":value,"Cases Fully Eligible":full_cases_buckets[key],"Records Fully Eligible":full_record_bucket[key]})
    

# for bucket in buckets:
#     finalanswer.append({'Bucket':bucket,'Amount':buckets[bucket]})
print(finalanswer)
df = pd.DataFrame(finalanswer)
#df['Amount'] = df['Amount'].astype(float)
#df['Bucket'] = df['Bucket'].astype(str)
display(df)
#legendarray = ["$100 or less","$250 or less", "$500 or less", "$1000 or less", "All"]
title = 'Charges and Cases Eligible for Expungement \nBut For Fines and Fees Owed'
temp = df.plot(kind="bar",x='Amount',ylabel="Amount of Charges, Cases, and Records",xlabel='Amount Owed',title=title)
temp.figure.savefig("Expungement Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


In [ ]:
#Charges/Cases with at least one charge/Full Cases Eligible for expungement but for fine and fee JUST OVER 70 ELIGIBLE
print("Charges and Cases Eligible But For Fine and Fee")

sql = '''SELECT *,COUNT(*) as charge_count FROM expungement WHERE 
DispositionCategory = 'Conviction' and Age >= 70 and TenYearArrestFreePeriod = 'True' and FinesAndFees > 0
GROUP BY DocketNumber'''

finalanswer = []

dockets = sendSQLCommand(sql)
charges_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
at_least_1_cases_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
full_cases_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
for docket in dockets:
    if(float(docket['FinesAndFees']) <= 100):
        charges_buckets['$100 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$100 or Less'] += 1
        else:
            at_least_1_cases_buckets['$100 or Less'] += 1

    if(float(docket['FinesAndFees']) <= 250):
        charges_buckets['$250 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$250 or Less'] += 1
        else:
            at_least_1_cases_buckets['$250 or Less'] += 1

    if(float(docket['FinesAndFees']) <= 500):
        charges_buckets['$500 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$500 or Less'] += 1
        else:
            at_least_1_cases_buckets['$500 or Less'] += 1
        
    if(float(docket['FinesAndFees']) <= 1000):
        charges_buckets['$1000 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$1000 or Less'] += 1
        else:
            at_least_1_cases_buckets['$1000 or Less'] += 1

    
    if(float(docket['FinesAndFees']) <= 5000):
        charges_buckets['$5000 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$5000 or Less'] += 1
        else:
            at_least_1_cases_buckets['$5000 or Less'] += 1
    
    charges_buckets["All"] += docket['charge_count']
    if(docket['charge_count'] == docket['TotalCharges']):
        full_cases_buckets['All'] += 1
    else:
        at_least_1_cases_buckets["All"] += 1


print(charges_buckets)
print(at_least_1_cases_buckets)
print(full_cases_buckets)

#Get Full Record
#Full Record Eligible But For Fee
sql = '''WITH temp AS 
(SELECT *, CASE 
WHEN 
DispositionCategory = 'Conviction' and Age >= 70 and TenYearArrestFreePeriod = 'True' and FinesAndFees > 0
THEN 1
ELSE 0
END AS Eligibility
FROM expungement) SELECT *, MIN(Eligibility) as CaseEligibility,IF(MIN(Eligibility) = 1,1,NULL) as `Record Fully Eligible` FROM temp GROUP BY PersonID
'''

finalanswer = []

dockets = sendSQLCommand(sql)
full_record_bucket = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}

for docket in dockets:
    if(docket['CaseEligibility'] == 1):
        if(float(docket['FinesAndFees']) <= 100):
            full_record_bucket['$100 or Less'] += 1

        if(float(docket['FinesAndFees']) <= 250):
            full_record_bucket['$250 or Less'] += 1


        if(float(docket['FinesAndFees']) <= 500):
            full_record_bucket['$500 or Less'] += 1

        if(float(docket['FinesAndFees']) <= 1000):
            full_record_bucket['$1000 or Less'] += 1


        if(float(docket['FinesAndFees']) <= 5000):
            full_record_bucket['$5000 or Less'] += 1

        full_record_bucket["All"] += 1






#Combine Both the Cases and Full Record Eligibility
for key,value in at_least_1_cases_buckets.items():
    finalanswer.append({"Amount":key,"Charges Eligible":charges_buckets[key],"Cases Partially Eligible":value,"Full Cases Eligible":full_cases_buckets[key],"Full Record Eligible":full_record_bucket[key]})
    

# for bucket in buckets:
#     finalanswer.append({'Bucket':bucket,'Amount':buckets[bucket]})
print(finalanswer)
df = pd.DataFrame(finalanswer)
#df['Amount'] = df['Amount'].astype(float)
#df['Bucket'] = df['Bucket'].astype(str)
display(df)
#legendarray = ["$100 or less","$250 or less", "$500 or less", "$1000 or less", "All"]
title = 'Charges and Cases Eligible for Expungement \nBut For Fines and Fees Owed'
temp = df.plot(kind="bar",x='Amount',ylabel="Amount of Charges, Cases, and Records",xlabel='Amount Owed',title=title)
#temp.figure.savefig("Expungement Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


In [ ]:
#Distinct People Over 70 Blocked By Fines and Fees
sql = '''SELECT COUNT(DISTINCT(PersonID)) FROM expungement WHERE DispositionCategory IN ('Conviction','Unknown') and 
Age >= 70 and TenYearArrestFreePeriod = 'True' and FinesAndFees > 0'''
dockets = sendSQLCommand(sql)
print(dockets)


#Total Charges For People Over 70 Blocked By Fines and Fees
sql = '''SELECT COUNT(*) FROM expungement WHERE 
DispositionCategory IN ('Conviction','Unknown') and Age >= 70 and TenYearArrestFreePeriod = 'True' and FinesAndFees > 0'''
dockets = sendSQLCommand(sql)
print(dockets)
